In [22]:
import sys
print(sys.executable)


/Users/nishapardeshi/Documents/Projects/youtube-comments-analyzer/youtube-comments-analyzer/.venv/bin/python3


In [23]:
import os
import re
from dotenv import load_dotenv
import gradio as gr
from googleapiclient.discovery import build
from typing import List, Dict, Optional
import openai

# Load environment variables
load_dotenv()

YOUTUBE_API_KEY = os.getenv("YOUTUBE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not YOUTUBE_API_KEY:
    raise ValueError("YOUTUBE_API_KEY not found in .env file")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY not found in .env file")

openai.api_key = OPENAI_API_KEY

# --- Helper functions ---

def extract_video_id(url: str) -> str:
    """
    Extract YouTube video ID from URL
    """
    pattern = r"(?:v=|youtu\.be/|youtube\.com/embed/)([A-Za-z0-9_-]{11})"
    match = re.search(pattern, url)
    if match:
        return match.group(1)
    raise ValueError("Invalid YouTube URL")


def get_youtube_comments(
    video_id: str,
    api_key: Optional[str] = None,
    max_comments: Optional[int] = 100
) -> List[Dict]:
    if api_key is None:
        api_key = YOUTUBE_API_KEY

    youtube = build("youtube", "v3", developerKey=api_key)
    comments: List[Dict] = []
    next_page_token: Optional[str] = None

    while True:
        request = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            maxResults=100,
            textFormat="plainText",
            pageToken=next_page_token,
            order="time",
        )
        response = request.execute()
        for item in response.get("items", []):
            snippet = item["snippet"]["topLevelComment"]["snippet"]
            comments.append(snippet.get("textDisplay", ""))
            if max_comments and len(comments) >= max_comments:
                return comments[:max_comments]

        next_page_token = response.get("nextPageToken")
        if not next_page_token:
            break

    return comments


def summarize_comments(comments: List[str]) -> str:
    """
    Use OpenAI to summarize YouTube comments
    """
    if not comments:
        return "No comments found."

    # System message defines role and context for the model
    system_message = {
        "role": "system",
        "content": "You are an expert sentiment analyst specializing in analyzing YouTube comments."
    }

    # User message includes instructions and the actual comments
    user_message = {
        "role": "user",
        "content": f"""
        I will provide a list of comments from a YouTube video. 
        Your task:

        1. Analyze the overall sentiment of the comments (positive, neutral, negative).
        2. Based on the sentiment, give a clear recommendation: Should a new viewer watch or skip the video?
        3. Provide a short explanation for your recommendation (1-3 sentences).

    Here is the list of comments:
    {comments}
    """
    }

   
    response = openai.chat.completions.create(
        model=MODEL,  # or "gpt-4o" depending on access
        messages=[system_message, user_message],
        temperature=0.7,
        max_tokens=1500
    )

    # Print the model's response
    # Correct way to get the content
    print(response.choices[0].message.content)

    summary = response.choices[0].message.content.strip()
    return summary


# --- Gradio interface ---

def analyze_video(url: str) -> str:
    try:
        video_id = extract_video_id(url)
        comments = get_youtube_comments(video_id)
        summary = summarize_comments(comments)
        return summary
    except Exception as e:
        return f"Error: {e}"





In [24]:
with gr.Blocks() as demo:
    gr.Markdown("## YouTube Comments Analyzer")
    gr.Markdown("Enter a YouTube URL to get a summary of the top comments.")

    url_input = gr.Textbox(label="YouTube URL", placeholder="https://www.youtube.com/watch?v=...")
    output_box = gr.Textbox(label="Comments Summary", lines=10)

    submit_btn = gr.Button("Analyze")
    submit_btn.click(analyze_video, inputs=url_input, outputs=output_box)

demo.launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.
